# What am I looking at?

This is Visual Studio Code.

We'll see how it works as we go.

In [ ]:
1 + 1

# Lab & Dictionary Recap

This list contains the email addresses of 4 different students

In [ ]:
emails = ["z1234567@student.unsw.edu.au", "z7654321@unsw.edu.au", "z7891234@ad.unsw.edu.au", "z1357924@student.unsw.edu.au"]

Can we build a dictionary containing the ZIDs as keys and email addresses as values?

How many students prefer each email domain?

Can we group students by their preferred email domain?

# Search algorithms

Search algorithms are a well studied class of algorithms as many problems can be expressed as search problems.

As an example, consider this "map" of Romania

<img src="https://d3i71xaburhd42.cloudfront.net/437af7588c6a36fd55c410b7f92b7f47ef57653b/5-Figure3.2-1.png" alt="Romania" />

### Depth-first search

In depth-first search, we explore as far as possible along each path before going back and finding another one.

See last week's lecture.

### Breadth-first search

In breadth-first search, we explore by moving outward from the start along all possible paths.

See last week's lecture

This is the map of Romania as a dictionary of dictionaries:

In [ ]:
romania = {
    'Arad': { 'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118 },
    'Zerind': { 'Arad': 75, 'Oradea': 71 },
    'Sibiu': { 'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80 },
    'Timisoara': { 'Arad': 118, 'Lugoj': 111 },
    'Oradea': { 'Zerind': 71, 'Sibiu': 151 },
    'Lugoj': { 'Timisoara': 111, 'Mehadia': 70 },
    'Fagaras': { 'Sibiu': 99, 'Bucharest': 211 },
    'Rimnicu Vilcea': { 'Sibiu': 80, 'Pitesti': 97, 'Craiova': 146 },
    'Mehadia': { 'Lugoj': 70, 'Dobreta': 75 },
    'Bucharest': { 'Fagaras': 211, 'Pitesti': 101, 'Urziceni': 85, 'Giurglu': 90 },
    'Pitesti': { 'Rimnicu Vilcea': 97, 'Bucharest': 101, 'Craiova': 138 },
    'Craiova': { 'Rimnicu Vilcea': 146, 'Pitesti': 138, 'Dobreta': 120 },
    'Dobreta': { 'Mehadia': 75, 'Craiova': 120 },
    'Urziceni': { 'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142 },
    'Giurglu': { 'Bucharest': 90 },
    'Hirsova': { 'Urziceni': 98, 'Eforie': 86 },
    'Vaslui': { 'Urziceni': 142, 'Lasi': 92 },
    'Eforie': { 'Hirsova': 86 },
    'Lasi': { 'Vaslui': 92, 'Neamt': 87 },
    'Neamt': { 'Lasi': 87 }
}

## Uniform cost search

In uniform-cost search, we explore by moving outward from the start, but exploring along the shortest path found so far first.

Find the shortest path from Arad to Bucharest using uniform-cost search.

# Serialisation

Serialisation is the process of translating data structures into a format such that they can be *stored* or *transmitted*.

Consider this list of strings

In [ ]:
example_strings = ["hello", "how are you?", "that's good", "nice to meet you"]

Can we store them in a text file?

Can we read them back from the file we stored them in?

What could go wrong with this appraoch?

## Comma Separated Values

Tabular (table-like) data can be stored in Comma Separated Value (CSV) format.

Load in the tutorial data from `tutorials.csv`.

Add a new tutorial then write the data back again.

Can we load in the tutorial data as a list of dictionaries?

Can we write it back from a list of dictionaries into CSV?